# Tavily Search Example with Answer

This notebook demonstrates how to use TavilySearchResults with the `include_answer` parameter to get both search results and answer in a single API call, with tool call invocation.

Reference: https://python.langchain.com/api_reference/_modules/langchain_community/tools/tavily_search/tool.html#TavilySearchResults

## Step 1: Import Required Libraries

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults

## Step 2: Load Environment Variables

In [2]:
# Load API key from .env file
load_dotenv()

# Verify API key is loaded
if os.getenv("TAVILY_API_KEY") is None:
    print("Tavily API key is not set in .env file")
    exit(1)
else:
    print("Tavily API key loaded successfully!")

Tavily API key loaded successfully!


## Step 3: Create Search Tool

In [3]:
# Create the search tool with include_answer=True
search_tool = TavilySearchResults(
    max_results=1,
    include_answer=True,  # This will include an answer in the response
    # include_raw_content=True,  # Optional: includes parsed HTML of search results
    # include_images=True,  # Optional: includes related images
)

print("Search tool created successfully!")

Search tool created successfully!


## Step 4: Execute Search Query

In [4]:
# Define the search query
query = "When will be the next canadian federal election?"

# Execute the search using tool call invocation
print("Executing search query...")
tool_response = search_tool.invoke({
    "args": {'query': query},
    "type": "tool_call",
    "id": "search_1",
    "name": "tavily"
})

print(f"Tool response type: {type(tool_response)}")
print(f"Tool response attributes: {dir(tool_response)[:10]}")

Executing search query...
Tool response type: <class 'langchain_core.messages.tool.ToolMessage'>
Tool response attributes: ['__abstractmethods__', '__add__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__']


## Step 5: Process and Display Results

In [5]:
# Access the artifact which contains all the detailed information
if hasattr(tool_response, 'artifact'):
    artifact = tool_response.artifact
    
    print("ARTIFACT CONTENTS:")
    print(f"Artifact type: {type(artifact)}")
    print(f"Artifact keys: {artifact.keys() if hasattr(artifact, 'keys') else 'Not a dict'}")
    
    # Print the answer from the artifact
    print("\nAnswer from artifact:")
    print(artifact.get("answer", "No answer found"))
    
    # Print the search results from the artifact
    print("\nSearch Results from artifact:")
    results = artifact.get("results", [])
    for i, result in enumerate(results, 1):
        print(f"{i}. {result.get('title', 'No title')}")
        print(f"   URL: {result.get('url', 'No URL')}")
        print(f"   Content: {result.get('content', 'No content')[:150]}...\n")
    
    # Print image URLs if available
    images = artifact.get("images", [])
    if images:
        print("\nImage URLs from artifact:")
        for i, image_url in enumerate(images, 1):
            print(f"{i}. {image_url}")
            
    # Print follow-up questions if available
    follow_up = artifact.get("follow_up_questions", None)
    if follow_up:
        print("\nFollow-up questions:")
        for i, question in enumerate(follow_up, 1):
            print(f"{i}. {question}")
            
    # Print response time
    print(f"\nResponse time: {artifact.get('response_time', 'N/A')} seconds")
else:
    print("No artifact found in the tool response")

ARTIFACT CONTENTS:
Artifact type: <class 'dict'>
Artifact keys: dict_keys(['query', 'follow_up_questions', 'answer', 'images', 'results', 'response_time'])

Answer from artifact:
The next Canadian federal election will be held on April 28, 2025. This will be the first election using a new electoral map based on the 2021 census. The writs of election were issued on March 23, 2025.

Search Results from artifact:
1. 2025 Canadian federal election - Wikipedia
   URL: https://en.wikipedia.org/wiki/2025_Canadian_federal_election
   Content: The 2025 Canadian federal election will be held on April 28, 2025, to elect members of the House of Commons to the 45th Canadian Parliament. The writs...


Response time: 1.68 seconds
